In [1]:
import torch
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import wandb

In [2]:
!python3 -m wandb login eb7b1964fb84cd81de96b2a273ecf2bb6254aeac

wandb: Appending key for api.wandb.ai to your netrc file: /home/alexeyorlov53/.netrc


In [3]:
filename = 'ecfp0'
samples_count1 = '10M'
model_name1 = f'molberto_{filename}_{samples_count1}'

In [4]:
batch_size = 1024

In [5]:
gpu_number = 2

In [6]:
target = 'NR-AR'

In [7]:
num_epoch = 10

In [8]:
lr = 0.001

### Upload and Split Dataset

In [9]:
dataframe = pd.read_csv("tox21-ecfp.csv")

In [10]:
def preprocess_data_dataset(df, column):
    for row in tqdm(range(len(df))):
        str_ints = eval(df.iloc[row][column])
        str_fingerprint = ' '.join(str_ints)
        df.at[row, column] = str_fingerprint

In [11]:
preprocess_data_dataset(dataframe, 'ecfp')

  0%|          | 0/22758 [00:00<?, ?it/s]

In [12]:
dataframe

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,Smiles,ecfp
0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,COc1ccc2c(c1OC)CN1CCc3cc4c(cc3C1C2)OCO4,2246728737 864674487 3217380708 3218693969 321...
1,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,COc1ccc2c(c1OC)CN1CCc3cc4c(cc3C1C2)OCO4,3975275337 2076190208 1135286194 951226070 951...
2,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,COc1ccc2c(c1OC)CN1CCc3cc4c(cc3C1C2)OCO4,932712697 252686949 589930590 3705139132 13780...
3,1.0,1.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...,2246728737 2976816164 2968968094 2968968094 29...
4,1.0,1.0,0.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1CC[C...,1861965050 778974661 2119439498 2117068077 172...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,CCN1C(=O)NC(c2ccccc2)C1=O,3542456614 2251845666 3346582092 3119383463 10...
22754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,CCN1C(=O)NC(c2ccccc2)C1=O,464037414 2602615753 2681514009 1799979616 845...
22755,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,2246728737 2245384272 864674487 3217380708 321...
22756,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,3542456614 3994088662 2115476908 3162837314 95...


In [13]:
dataframe = dataframe.dropna(subset=[target]).reset_index(drop=True)

In [14]:
# dataframe = dataframe[0:3500]

In [15]:
# print('Percentage on NaNs:')
# dataframe.isna().mean()

In [16]:
# rows_with_nans = dataframe['Molecular Weight'].isna() | \
#                  dataframe['Bioactivities'].isna() | \
#                  dataframe['AlogP'].isna() | \
#                  dataframe['Polar Surface Area'].isna() | \
#                  dataframe['CX Acidic pKa'].isna() | \
#                  dataframe['CX Basic pKa'].isna()
# print(f'Count of rows without NaNs: {dataframe.shape[0] - dataframe.loc[rows_with_nans].shape[0]}')

In [17]:
# remove 2 last properties to reduce NaN counts
# molecular_properties = ['Molecular Weight', 'Bioactivities', 'AlogP', 'Polar Surface Area']
# dataframe = dataframe.drop(columns=['CX Acidic pKa', 'CX Basic pKa'])

In [18]:
# drop NaN's
# dataframe = dataframe.dropna().reset_index(drop=True)

In [19]:
# dataframe

In [20]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(dataframe)
train_testvalid = dataset.train_test_split(test_size=0.2, seed=15)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=15)

# 10% for test, 10 for validation, 80% for train
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

dataset

DatasetDict({
    train: Dataset({
        features: ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53', 'Smiles', 'ecfp'],
        num_rows: 16876
    })
    test: Dataset({
        features: ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53', 'Smiles', 'ecfp'],
        num_rows: 2110
    })
    validation: Dataset({
        features: ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53', 'Smiles', 'ecfp'],
        num_rows: 2110
    })
})

### Tokenize Data

In [21]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name1)

tokenizer.model_max_len=512

In [22]:
def tokenize(batch):
  return tokenizer(batch["ecfp"], truncation=True, max_length=512, padding='max_length')

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/16876 [00:00<?, ? examples/s]

Map:   0%|          | 0/2110 [00:00<?, ? examples/s]

Map:   0%|          | 0/2110 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53', 'Smiles', 'ecfp', 'input_ids', 'attention_mask'],
        num_rows: 16876
    })
    test: Dataset({
        features: ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53', 'Smiles', 'ecfp', 'input_ids', 'attention_mask'],
        num_rows: 2110
    })
    validation: Dataset({
        features: ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53', 'Smiles', 'ecfp', 'input_ids', 'attention_mask'],
        num_rows: 2110
    })
})

In [23]:
columns = ["input_ids", "attention_mask"]
columns.extend([target]) # our labels
print(columns)
tokenized_dataset.set_format('torch', columns=columns)

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

['input_ids', 'attention_mask', 'NR-AR']


### Create Transformer Model

In [24]:
from transformers import AutoModel, AutoConfig

class MolecularPropertiesClassification(torch.nn.Module):
    def __init__(self, model_name1):
        super(MolecularPropertiesClassification, self).__init__()
        
        config1 = AutoConfig.from_pretrained(model_name1)
        self.transformer1 = AutoModel.from_pretrained(model_name1, config=config1)
        # removing last layer of transformer
        self.transformer1.pooler = torch.nn.Identity()
        # freezing transformer weights
        for param in self.transformer1.parameters():
            param.requires_grad = False

        self.linear1 = torch.nn.Linear(768, 768, bias=True)
        self.linear2 = torch.nn.Linear(768, 2, bias=True)

    def forward(self, input_ids = None, attention_mask=None):
        outputs1 = self.transformer1(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state1 = outputs1[0]
        
        first_linear_out = self.linear1(last_hidden_state1[:, 0, : ].view(-1, 768))
        logits = self.linear2(torch.nn.functional.sigmoid(first_linear_out))

        return logits
        

### Create PyTorch DataLoader

In [25]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset['train'], shuffle = True, batch_size = batch_size, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    tokenized_dataset['validation'], shuffle = True, batch_size = batch_size, collate_fn = data_collator
)

test_dataloader = DataLoader(
    tokenized_dataset['test'], shuffle = True, batch_size = batch_size, collate_fn = data_collator
)

In [26]:
device = torch.device("cuda", index=gpu_number) if torch.cuda.is_available() else torch.device('cpu')

model = MolecularPropertiesClassification(model_name1).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at molberto_ecfp0_10M and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
model

MolecularPropertiesClassification(
  (transformer1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [28]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=lr)

num_training_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

loss_func = torch.nn.CrossEntropyLoss()

/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [29]:
wandb.init(
    project="efcp_transformer",
    name='ECFP-BERT-' + samples_count1 + "-Tox21",
    config={}
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: orlov-aleksei53 (moleculary-ai). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible


### Training

In [30]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epoch * len(eval_dataloader)))

for epoch in range(num_epoch):
    model.train()
    total_pred_labels = []
    total_true_labels = []
    epoch_loss = 0
    for batch in train_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        batch[target] = batch[target].to(device)
        
        logits = model(**input_batch)
        
        loss = loss_func(logits.view(-1, 2), batch[target].view(-1).type(torch.cuda.LongTensor))
        loss.backward()
        epoch_loss += loss.item()
        
        pred_labels = torch.argmax(logits, dim=-1)
        true_labels = batch[target]

        print(pred_labels)
        print(true_labels)
        total_pred_labels.append(pred_labels)
        total_true_labels.append(true_labels)
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

    total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
    total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()

    train_loss = epoch_loss / len(train_dataloader)
    train_accuracy = accuracy_score(total_true_labels, total_pred_labels)
    train_f1_score = f1_score(total_true_labels, total_pred_labels, average='micro')
    train_precision = precision_score(total_true_labels, total_pred_labels, average='micro')
    train_recall = recall_score(total_true_labels, total_pred_labels, average='micro')
    train_roc_auc = roc_auc_score(total_true_labels, total_pred_labels)
    
    
    model.eval()
    total_pred_labels = []
    total_true_labels = []
    epoch_loss = 0
    for batch in eval_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        batch[target] = batch[target].to(device)
        
        with torch.no_grad():
            logits = model(**input_batch)
            loss = loss_func(logits.view(-1, 2), batch[target].view(-1).type(torch.cuda.LongTensor))
            epoch_loss += loss.item()

            pred_labels = torch.argmax(logits, dim=-1)
            true_labels = batch[target]
            total_pred_labels.append(pred_labels)
            total_true_labels.append(true_labels)
        
        progress_bar_eval.update(1)

    total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
    total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()

    validation_loss = epoch_loss / len(eval_dataloader)
    validation_accuracy = accuracy_score(total_true_labels, total_pred_labels)
    validation_f1_score = f1_score(total_true_labels, total_pred_labels, average='micro')
    validation_precision = precision_score(total_true_labels, total_pred_labels, average='micro')
    validation_recall = recall_score(total_true_labels, total_pred_labels, average='micro')
    validation_roc_auc = roc_auc_score(total_true_labels, total_pred_labels)

    wandb.log({"loss/train": train_loss, 
               "accuracy/train": train_accuracy, 
               "f1/train": train_f1_score,
               "precision/train": train_precision, 
               "recall/train": train_recall, 
               "roc_auc_score/train": train_roc_auc,
               
               "loss/validation": validation_loss, 
               "accuracy/validation": validation_accuracy, 
               "f1/validation": validation_f1_score, 
               "precision/validation": validation_precision, 
               "recall/validation": validation_recall, 
               "roc_auc_score/validation": validation_roc_auc})

  0%|          | 0/170 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
def test_loop():
    model.eval()
    total_pred_labels = []
    total_true_labels = []
    epoch_loss = 0
    for batch in test_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        batch[target] = batch[target].to(device)
        
        with torch.no_grad():
            logits = model(**input_batch)
            loss = loss_func(logits.view(-1, 2), batch[target].view(-1).type(torch.cuda.LongTensor))
            epoch_loss += loss.item()

            pred_labels = torch.argmax(logits, dim=-1)
            true_labels = batch[target]
            total_pred_labels.append(pred_labels)
            total_true_labels.append(true_labels)
        
        progress_bar_eval.update(1)

    total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
    total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()
    
    wandb.log({"loss/test": epoch_loss / len(test_dataloader)})
    wandb.log({"accuracy/test": accuracy_score(total_true_labels, total_pred_labels)})
    wandb.log({"f1/test": f1_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"precision/test": precision_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"recall/test": recall_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"roc_auc_score/test": roc_auc_score(total_true_labels, total_pred_labels)})

In [ ]:
test_loop()

In [ ]:
wandb.finish()

In [ ]:
torch.cuda.empty_cache()